In [ ]:
# crawl San Francisco top 100 donut shop's information from yelp
# fileds:rank, name, address(geolocation), website_url,rating, type tag, delivery/dine in tag, number of reviews, if can be ordered on yelp
# clean and form all the data and then store them into MongoDB for later analysis

In [27]:
from bs4 import BeautifulSoup
import requests
import time
import re
import random
from pymongo import MongoClient
import pandas as pd

In [ ]:
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
"authority": "www.tagesschau.de",
"method": "GET",
"path":"/",
"scheme":"https",
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9,de;q=0.8",
"cache-control": "max-age=0",
"cookie": "atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2257ea5dd6-4c35-4982-942f-8a7f8b8c3a4b%22%2C%22options%22%3A%7B%22end%22%3A%222023-02-17T05%3A02%3A59.936Z%22%2C%22path%22%3A%22%2F%22%7D%7D; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-595936-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D",
"sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": "Windows",
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1"}

# initiate list
rank_list = []
name_list = []
url_list = []
rating_list = []
reviews_list = []
tag_list = []
sign_list = []
delivery_tag_list = []
order_yelp_list = []
address_list = []
phone_list = []
website_list = []
geo_list = []

# Get top 100's search result page
for ii in range(0,100,10):
    URL = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&ns=1&start="+str(ii)
    page = requests.get(URL, headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    all = soup.find_all('div',attrs={'class':'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'})
    # Get non-sponsor shop
    for i in range(2,12):
        rank_name_url_all = all[i].find_all('span',attrs={'class':'css-1uq0cfn'})[0]
        # rank and name
        rank_name = rank_name_url_all.get_text()
        rank_list.append(re.match('([0-9].*?)\.(.*)',rank_name).group(1))
        name_list.append(re.match('([0-9].*?)\.(.*)',rank_name).group(2))
        
        # url
        url_list.append('https://www.yelp.com'+rank_name_url_all.select('a')[0].get('href'))

        # rating
        rating = all[i].find('span',attrs={'class':'display--inline__09f24__c6N_k border-color--default__09f24__NPAKY'}).select('div')[0].get('aria-label')
        rating_list.append(re.match('(.*)( star rating)',rating).group(1))

        # num of reviews
        reviews_list.append(all[i].find('span',attrs={'class':'reviewCount__09f24__tnBk4 css-1e4fdj9'}).text)

        # tags
        tags = all[i].find_all('button',attrs={'class':'css-un1vt9'})
        sub_tag = []
        for tag in tags:
            sub_tag.append(tag.get_text())
        tag_list.append(sub_tag)

        # dollar sign
        if all[i].find('span',attrs={'class':'priceRange__09f24__mmOuH css-18qxe2r'}):
            sign = all[i].find('span',attrs={'class':'priceRange__09f24__mmOuH css-18qxe2r'}).get_text()
            sign_list.append(sign)
        else:
            sign_list.append('NULL')

        # delivery tags
        signal1 = all[i].find_all('div',attrs={'class':'tag__09f24__mjBHZ css-1v994a0 margin-t0-5__09f24__gboxT border-color--default__09f24__NPAKY'})
        tmp = {}
        if signal1:
            for sub in signal1:
                signal2 = sub.find('path',attrs={'d':'M9.06 8l3.47-3.47a.75.75 0 00-1.06-1.06L8 6.94 4.53 3.47a.75.75 0 00-1.06 1.06L6.94 8l-3.47 3.47a.75.75 0 101.06 1.06L8 9.06l3.47 3.47a.75.75 0 001.06-1.06L9.06 8z'})
                if signal2:
                    tmp[sub.get_text()] = 0
                else:
                    tmp[sub.get_text()] = 1
            delivery_tag_list.append(tmp)
        else:
            delivery_tag_list.append('NULL')

        # if u can order through yelp
        order_signal = all[i].find('span',attrs={'class':'css-1enow5j'})
        if order_signal:
            order_yelp_list.append('yes')
        else:
            order_yelp_list.append('no')
        
        # time out
        wait_time = random.randrange(5,8)
        time.sleep(wait_time)


# Click into each shop and get their address, phone number and webiste
for url in url_list:
    page = requests.get(url, headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')

    whole = soup.find('div',attrs={'class':'css-xp8w2v padding-t2__09f24__Y6duA padding-r2__09f24__ByXi4 padding-b2__09f24__F0z5y padding-l2__09f24__kf_t_ border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-radius--regular__09f24__MLlCO background-color--white__09f24__ulvSM'})
    web_phone = soup.find_all('p',attrs={'class':'css-na3oda'})
    if len(web_phone) == 2:
        website_list.append(whole.find_all('p',attrs={'class':'css-1p9ibgf'})[0].text)
        phone_list.append(whole.find_all('p',attrs={'class':'css-1p9ibgf'})[1].text)
    elif len(web_phone) == 1:
        if web_phone[0].text == 'Business website':
            website_list.append(whole.find('p',attrs={'class':'css-1p9ibgf'}).text)
            phone_list.append('NULL')
        else:
            website_list.append('NULL')
            phone_list.append(whole.find('p',attrs={'class':'css-1p9ibgf'}).text)
    else:
        website_list.append('NULL')
        phone_list.append('NULL')

    address_list.append(soup.find('p',attrs={'class':'css-qyp8bo'}).get_text())
    

# Use API to get each shop's longitude and latitude   
for address in address_list:
    url_api = 'http://api.positionstack.com/v1/forward?access_key=88fea48dfe7281c431ba2b922f1ed775&query='+address
    page = requests.get(url_api, headers=header)
    return_data = page.json()
    
    latitude = return_data['data'][0]['latitude']
    longitude = return_data['data'][0]['longitude']
    
    geo = (longitude,latitude)
    
    geo_list.append(geo)

In [ ]:
# store all the information into MongoDB
# make connection
client = MongoClient('localhost', 27017)
# create yelp database 
yelp_db = client['yelp_db']
# create colletion in this database
donut_shop_col = yelp_db['sf_donut_shops']

for i in range(0,40):
    info = {
        'rank':rank_list[i],
        'name':name_list[i],
        'address':address_list[i],
        'phone number':phone_list[i],
        'url':url_list[i],
        'website':website_list[i],
        'rating':rating_list[i],
        'number of reviews':reviews_list[i],
        'tag':tag_list[i],
        'how many $ sign':sign_list[i],
        'delivery tag etc.':delivery_tag_list[i],
        'whether can be ordered on yelp':order_yelp_list[i],
        'geolocation': geo_list[i]
    }
    donut_shop_col.insert_one(info)

if 'yelp' in client.list_database_names():
    print('Yelp database has been created!')
    
if 'sf_donut_shops' in yelp_db.list_collection_names():
    print("Donut shop collection has been created!")

In [ ]:
data = {'rank':rank_list,
        'name':name_list,
        'address':address_list,
        'phone number':phone_list,
        'url':url_list,
        'website':website_list,
        'rating':rating_list,
        'number of reviews':reviews_list,
        'tag':tag_list,
        'how many $ sign':sign_list,
        'delivery tag etc.':delivery_tag_list,
        'whether can be ordered on yelp':order_yelp_list,
        'geolocation': geo_list}
df = pd.DataFrame(data)
df